## 关于自顶向上和自底向下

这两个的区别其实很简单：只要判断是先检测人再在检测到的人上做单人姿态估计，还是先检测帧内的所有关键点，再进行关键点的分配即可。有一个方法相对比较特殊，2018年ECCV的multiposenet（首页上我给出了文章和我复现的代码的链接），作者将关键点检测和人的bounding box检测放在两个子网络中进行，最后用检测到的人的bounding box信息对人的空间位置做了约束。

至于为什么出现了这两种方法，其实两张图就可以说明问题，首先说明为什么有自底向上这种方法存在：快。毕竟每个人都做一次pose的估计还是很浪费时间的，因此估计的时间肯定和人数是成正比的。我们可以看一张定量的性能图：

![avatar](../imgs/speed.png)

图中的红色部分CMU-pose就是Realtime Multi-Person 2D Pose Estimation using Part Affinity Fields中利用paf编码空间相对信息的方法，蓝色曲线our就是2018 ECCV的multiposenet的方法，backbone是multiposenet前面的backbone net，具体可以看文章和我的repo的代码。Top-down的绿色曲线是Top-down方法随人数增长的runtime增长，可以看出在时间开销上，自底向上是具有很大优势的。至于为什么蓝色曲线也会随着人数上升而上升，因为multiposenet的子网络也检测了人，所以有一定开销，但是检测人的开销比每个人单独做姿态估计小很多，多人的proposal比多个人的pose估计要快很多，所以运行时间上的消耗不是很大。但是自顶向下当然也是有优势的，精度摆在那。下面是目前（18年8月）为止一些主流方法在coco上的性能：

![avatar](../imgs/acc.png)

BU是bottom-up自底向上的方法，TD是top-down自顶向下的方法，精度差距还是蛮大的。毕竟目前检测器性能在上升，而且类似于图中SJTU那篇的方法还加了后续模块，加NMS等方法去除冗余，再结合目前已经比较成熟，精准的单人姿态估计方法，取得好性能也就很正常了。整体来说两种思路就是时间开销和估计精度的取舍，但是目前自底向上方法还有很大的潜力可挖。